# `CLOUD.jl` - 2D linear advection example

In [28]:
using OrdinaryDiffEq
using CLOUD;

Define the physical problem

In [29]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period
initial_data = InitialDataSine(A,k)
conservation_law = linear_advection_equation((a*cos(θ),a*sin(θ)),λ=1.0);

Set discretization parameters

In [30]:
M = 4
p = 4
p_map = 4
form = SplitConservationForm()
integrator = CarpenterKennedy2N54()
dt_scale=0.05
dt = dt_scale*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [31]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=10)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

visualize(spatial_discretization, results_path, "mesh.pdf", 
    plot_volume_nodes=false, grid_lines=true)

Run the solver and plot the solution

In [32]:
ode_problem = semidiscretize(load_project(results_path)...)
save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, integrator, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")
plotter = Plotter(spatial_discretization, results_path)
visualize(last(sol.u), plotter, "approx.pdf", u_range=[-1.0,1.0], contours=25,
    label="U^h(\\mathbf{x},t)")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:         85.4μs / 2932803.4%      11.9KiB / 21517358.2%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          81.5k    1.02s   40.8%  12.5μs    774MiB   31.1%  9.72KiB
   volume terms         81.5k    626ms   25.0%  7.68μs    500MiB   20.1%  6.28KiB
   facet terms          81.5k    191ms    7.6%  2.35μs    173MiB    6.9%  2.17KiB
   mass matrix solve    81.5k   92.4ms    3.7%  1.13μs   39.8MiB    1.6%     512B
 eval flux diff         81.5k    942ms   37.6%  11.6μs   0.99GiB   40.7%  12.7KiB
 extrapolate solution   81.5k    202ms    8.1%  2.48μs    142MiB    5.7%  1.78KiB
 eval numerical

 10.5%  3.28KiB
 ────────────────────────────────────────────────────────────────────────────────


Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [33]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.000683954884588324
Conservation (initial/final/diff):
[-1.108594540819019e-8][-1.1085945919933615e-8][-5.117434254131581e-16]
Energy (initial/final/diff):
[0.12499998811286066][0.12497956180878655][-2.0426304074103174e-5]


Set up a `DGMulti` scheme on triangles of degree `p` with `M` edges in each direction

In [34]:
reference_approximation = ReferenceApproximation(
    DGMulti(p), Tri(), mapping_degree=p_map, N_plot=10)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgmulti/", overwrite=true, clear=true)

visualize(spatial_discretization, results_path, "mesh.pdf", 
    plot_volume_nodes=true, grid_lines=false)

Run the solver and plot the solution

In [35]:
ode_problem = semidiscretize(load_project(results_path)...)

save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, integrator, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

plotter = Plotter(spatial_discretization, results_path)
visualize(last(sol.u), plotter, "approx.pdf", u_range=[-1.0,1.0], contours=25,
    label="U^h(\\mathbf{x},t)")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:         98.5μs / 5450015.1%      11.9KiB / 43636281.0%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual           163k    2.61s   48.7%  16.0μs   2.40GiB   48.6%  15.4KiB
   volume terms          163k    1.74s   32.4%  10.7μs   1.92GiB   38.8%  12.3KiB
   mass matrix solve     163k    525ms    9.8%  3.22μs    371MiB    7.3%  2.33KiB
   facet terms           163k   83.5ms    1.6%   512ns   27.4MiB    0.5%     176B
 eval flux diff          163k    1.64s   30.5%  10.1μs   1.33GiB   26.9%  8.55KiB
 eval numerical flux     163k    322ms    6.0%  1.98μs    333MiB    6.6%  2.09KiB
 eval flux     

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [36]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.0034367122388651375
Conservation (initial/final/diff):
[-1.4530229013823975e-7][-1.4530229025620095e-7][-1.1796119636642288e-16]
Energy (initial/final/diff):
[0.12499475017905026][0.12485504074741147][-0.0001397094316387959]
